# Explain the concept of tidy data
Reference paper: [Tidy Data](https://vita.had.co.nz/papers/tidy-data.pdf)
Reference DataCamp course: Reshaping data with pandas

Definition: 

In tidy data:
- Each variable forms a column
- Each observation forms a row
- Each type of observational unit forms a table

The five most common problems with messy datasets:
- Column headers are values, not variable names.
- Multiple variables are stored in one column.
- Variables are stored in both rows and columns.
- Multiple types of observational units are stored in the same table. 
- A single observational unit is stored in multiple tables.

Let's check them out!

# Transform the messy data into tidy data using Python

## Column headers are values, not variable names

For this particular example, we will recreate the original table from the original dataset of the Tidy Data paper

### Importing original dataset and extracting rows

In [34]:
# Original dataset http://pewforum.org/Datasets/Dataset-Download.aspx
# Sampple SPSS dataset from https://github.com/hadley/tidy-data/blob/master/data/pew.sav

import pandas as pd
import requests
import os.path

# pd.read_spss supports only file paths, so we need to download the file first
if os.path.exists('pew.sav'):
    print('File exists, skipping download')
else:
    response = requests.get('https://raw.github.com/hadley/tidy-data/master/data/pew.sav')
    response.raise_for_status()
    with open('pew.sav', 'wb') as file:
        file.write(response.content)
# To get the file directly, we need to access just the raw file, instead of the github page that serves the file. 
# Ref: https://stackoverflow.com/questions/14120502/how-to-download-and-write-a-file-from-github-using-requests

pew_df = pd.read_spss('pew.sav')
pew_df.head()

File exists, skipping download


,weight,psraid,int_date,lang,type,cregion,state,usr,usr1,form,...,q63,educ,income,regist,regicert,party,partyln,ideo,pvote04a,pvote04b
0,4.512821,10000001.0,50807.0,English,RDD,Northeast,Connecticut,Suburban,Suburban,Form A,...,"Yes, father born outside U.S.","Technical, trade, or vocational school AFTER h...","75 to under $100,000","Yes, registered",Absolutely certain,Republican,NaN,Moderate,Voted,Bush
1,2.102564,10000002.0,50807.0,English,RDD,Northeast,Maine,Rural,Rural,Form B,...,"No, both parents born in U.S.",High school graduate (Grade 12 or GED certific...,"20 to under $30,000","No, not registered",NaN,Republican,NaN,Conservative,Did not vote (includes too young to vote),NaN
2,1.282051,10000003.0,50807.0,English,RDD,Northeast,Maine,Rural,Rural,Form A,...,"No, both parents born in U.S.","College graduate (B.S., B.A., or other 4-year ...","30 to under $40,000","No, not registered",NaN,Independent,Democrat,Conservative,Did not vote (includes too young to vote),NaN
3,1.355323,10000004.0,50807.0,English,RDD,Northeast,Maine,Rural,Rural,Form B,...,"No, both parents born in U.S.","Some college, no 4-year degree (including asso...","Less than $10,000","No, not registered",NaN,Independent,Democrat,Moderate,Did not vote (includes too young to vote),NaN
4,1.589744,10000005.0,50807.0,English,RDD,Northeast,New York,Urban,Urban,Form A,...,"Yes, father born outside U.S.",Post-graduate training or professional schooli...,"50 to under $75,000","Yes, registered",Absolutely certain,Independent,Democrat,Moderate,Voted,Other candidate


In [35]:
# We extract these particular columns to create the table
# q16: religious beliefs?
# reltrad: religious tradition
# income: income in USD$
religion = pew_df[['q16', 'reltrad', 'income']]
religion.head()

,q16,reltrad,income
0,Protestant,Evangelical Protestant Churches,"75 to under $100,000"
1,Protestant,Mainline Protestant Churches,"20 to under $30,000"
2,Protestant,Mainline Protestant Churches,"30 to under $40,000"
3,Nothing in particular,Unaffiliated,"Less than $10,000"
4,Jewish (Judaism),Jewish,"50 to under $75,000"


In [36]:
religion['q16'].value_counts()

 Protestant                               18353
 Roman Catholic                            8047
 Nothing in particular                     3707
 Agnostic (not sure if there is a God)      826
 Jewish (Judaism)                           682
                                          ...  
 Satanism (VOL)                               1
 Unification Church (Moonies) (VOL)           1
 Maronite Catholic (VOL)                      1
 Animism (VOL)                                1
 Old Catholic (VOL)                           1
Name: q16, Length: 65, dtype: int64

### Data cleaning

In [37]:
# TODO Original versions return SettingWithCopyWarning messages
# Looking for ways to replace them for .loc versions
# All these transformation appear on original Tidy Data paper
# religion['reltrad'] = religion['reltrad'].str.replace(' Protestant', ' Prot') 
# religion['reltrad'] = religion['reltrad'].str.replace(' Churches', '')
religion["reltrad"] = (
    religion["reltrad"].str.replace(" Protestant", " Prot").str.replace(" Churches", "") 
)

/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_28734/3258246849.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion["reltrad"] = (


#### Warning
<div class="alert alert-block alert-warning">
<b>df.replace()</b>: replaces the content of a whole cell. It supports inplace=True argument. <br>
<b>df.str.replace()</b>: replaces fragments of strings in a cell. It doesn't support inplace=True argument.
</div>

In [38]:
# Assign a value for reltrad based on the q16 contents
# R versions: religion$reltrad[religion$q16 == " Atheist (do not believe in God) "] <- "Atheist"
# R versions: religion$reltrad[religion$q16 == " Agnostic (not sure if there is a God) "] <- "Agnostic"

# Note that in original example, there are leading and trailing whitespaces. Somehow ending whitespace was trimmed on import
# TODO Report a bug to pandas?

# So we match without using the final space!

# A direct translation of R code would be:
# religion.loc[religion['q16'].str.match(' Atheist (do not believe in God)'), 'reltrad'] = 'Atheist'
# religion.loc[religion['q16'].str.match(' Agnostic (not sure if there is a God)'), 'reltrad'] = 'Agnostic'
# However, in Python we need to escape parenthesis, or it will return zero matches.
# religion.loc[religion['q16'].str.match(' Atheist \(do not believe in God\)'), 'reltrad'] = 'Atheist'
# religion.loc[religion['q16'].str.match(' Agnostic \(not sure if there is a God\)'), 'reltrad'] = 'Agnostic'
# To simplify the string matching, we will use str.contains instead of match

religion.loc[religion['q16'].str.contains('Atheist'), 'reltrad'] = 'Atheist'
religion.loc[religion['q16'].str.contains('Agnostic'), 'reltrad'] = 'Agnostic'

# Alternative, traditional string comparison
# religion.loc[religion['q16'] == (' Agnostic (not sure if there is a God)'), 'reltrad'] = 'Agnostic'


/Users/cesargarciasaez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


#### Testing cells for matching embedded parenthesis in strings

In [39]:
num_prots = sum(religion['q16'].str.match(' Protestant'))
print(num_prots)

18353


In [40]:
num_agn = sum(religion['q16'].str.match(' Agnostic \(not sure if there is a God\)'))
print(num_agn)

826


In [41]:
# TODO Not working with this code
# religion['reltrad'].str.replace(r'\([^)]*\)', '', regex=True)
# religion['reltrad'].str.strip()

# Remove (expressions inside parenthesis) in reltrad
religion['reltrad'] = religion['reltrad'].str.replace(r'\([^)]*\)', '', regex=True)

# Trim spaces in reltrad: R version religion$reltrad <- str_trim(religion$reltrad)
religion['reltrad'] = religion['reltrad'].str.strip()

/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_28734/3148514731.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['reltrad'] = religion['reltrad'].str.replace(r'\([^)]*\)', '', regex=True)
/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_28734/3148514731.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['reltrad'] = religion['reltrad'].str.strip()


In [42]:
# Replace income values for shorter versions. I will use using map in this particular case. It seems to be more efficient! 
# Ref: https://stackoverflow.com/questions/42012339/using-replace-efficiently-in-pandas
# Note: income is already categorical data, so there is no need for binning actual incomes. These are already bins!

income_to_shorter_description = {
  "Less than $10,000": "<$10k", 
  "10 to under $20,000": "$10-20k", 
  "20 to under $30,000": "$20-30k", 
  "30 to under $40,000": "$30-40k", 
  "40 to under $50,000": "$40-50k", 
  "50 to under $75,000": "$50-75k",
  "75 to under $100,000": "$75-100k", 
  "100 to under $150,000": "$100-150k", 
  "$150,000 or more": ">150k", 
  "Don't know/Refused (VOL)": "Don't know/refused"
}

religion['income'] = religion['income'].map(income_to_shorter_description)
religion.head()

/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_28734/2322833425.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['income'] = religion['income'].map(income_to_shorter_description)


,q16,reltrad,income
0,Protestant,Evangelical Prot,$75-100k
1,Protestant,Mainline Prot,$20-30k
2,Protestant,Mainline Prot,$30-40k
3,Nothing in particular,Unaffiliated,<$10k
4,Jewish (Judaism),Jewish,$50-75k


### Reconstructing original table

In [43]:
# Now it's time to sort how many people are in each income group for each reltrad
# We will keep the original variable names here

counts = religion.groupby(['reltrad', 'income']).size()
# size() returns a Series with multi-index
# count() returns a Dataframe with multi-index and a column

In [44]:
# Reltrad appears as religion in the Tidy Data paper, so we have to rename the index
counts.index.set_names({'reltrad': 'religion'}, inplace=True)
counts.sort_index()
counts.index

MultiIndex([(             'Agnostic',   '$10-20k'),
            (             'Agnostic', '$100-150k'),
            (             'Agnostic',   '$20-30k'),
            (             'Agnostic',   '$30-40k'),
            (             'Agnostic',   '$40-50k'),
            (             'Agnostic',   '$50-75k'),
            (             'Agnostic',  '$75-100k'),
            (             'Agnostic',     '<$10k'),
            (             'Agnostic',     '>150k'),
            (              'Atheist',   '$10-20k'),
            ...
            ('Other World Religions',     '>150k'),
            (         'Unaffiliated',   '$10-20k'),
            (         'Unaffiliated', '$100-150k'),
            (         'Unaffiliated',   '$20-30k'),
            (         'Unaffiliated',   '$30-40k'),
            (         'Unaffiliated',   '$40-50k'),
            (         'Unaffiliated',   '$50-75k'),
            (         'Unaffiliated',  '$75-100k'),
            (         'Unaffiliated',     '<$10k

In [45]:
# This is our final destination, where we have been able to reproduce the table in the original paper.
# At the same time, this is the first step to start tidyng the data!
counts.unstack().head(10)

income,$10-20k,$100-150k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,<$10k,>150k
religion,,,,,,,,,
Agnostic,34,109,60,81,76,137,122,27,84
Atheist,27,59,37,52,35,70,73,12,74
Buddhist,21,39,30,34,33,58,62,27,53
Catholic,617,792,732,670,638,1116,949,418,633
Don’t know/refused,14,17,15,11,10,35,21,15,18
Evangelical Prot,869,723,1064,982,881,1486,949,575,414
Hindu,9,48,7,9,11,34,47,1,54
Historically Black Prot,244,81,236,238,197,223,131,228,78
Jehovah's Witness,27,11,24,24,21,30,15,20,6


### How would I do it if I had to start from scratch?
I would change the order of the cleaning operations to simplify the process. For example:

#### I would strip leading and trailing whitespaces in all the columns first

All reltrad rows start by a white space. In the original R file, they also ended by a white space. In the version I read, the ending white space is missing.

He matches the string with spaces, and then later removes all spaces. I think it is more clear and less error prone to strip white spaces first and then do the string matching/replacement.

White spaces can haunt you in so many unexpected places!
```python
MultiIndex([(' Buddhist',   '$10-20k'),
            (' Buddhist', '$100-150k'),
            (' Buddhist',   '$20-30k'),
            (' Buddhist',   '$30-40k'),
            (' Buddhist',   '$40-50k'),
            (' Buddhist',   '$50-75k'),
            (' Buddhist',  '$75-100k'),
            (' Buddhist',     '<$10k'),
            (' Buddhist',     '>150k'),
            (' Catholic',   '$10-20k'),
            ...
            ( 'Agnostic',     '>150k'),
            (  'Atheist',   '$10-20k'),
```

Did you see the whitespace before Buddhist scrambles any sort?

#### I would remove information under parenthesis earlier
If there information between parenthesis is not needed and doesn't offer any additional insights, it could make sense to remove them as early as possible, to make the output more readable and visible.

#### I would start creating softer string matches and then do stronger ones
I would start using str.contains() and then test with str.match(). I have spent too many hours figuring out why I could not match an expression that included whitespaces and parenthesis!

### Further research
- [ ] TODO I need to make a table with operations that require inplace=True, and those that will change the dataframe inplace by default.
- [ ] TODO I need to remove all the SettingWithCopyWarning errors. Example: religion['reltrad'] = religion['reltrad'].str.strip() (religion['reltrad'].str.strip() would not modify the original values unless this assignation is specified)
- [ ] TODO In the multi-index above, the categories are not sorted properly by income. Right now it is sorted like: $10k, $100k, $20k, [...], < $10k, > $10k.
- [ ] TODO The "Don't know / refused" category is missing from the output. In the paper, they mention that they have filtered some rows, but we haven't done that in our code AFAIK.
- [ ] Check what is the exact definition of melting in Python vs R. Establish clear criteria for transformation based on the concept of *colvars*

### Melting the dataframe to produce tidy data output
In order to produce tidy data format, we have to make sure that every row is a single observation, and that each column is a variable. These are called **colvars**.

When we melt, we need to pass the list of columns that are already variables, plus a list of other columns that will be converted into variables.

In this particular case, we have just a colvar, called *religion*. As in the original paper, we will rename the value column to *freq*. Variable column is already called *income*.

Please also note that most of the following steps are just unnecesary, as the *counts* dataframe created above as a multi-index series could be already be considered tidy data. We are doing the next steps for the sake of completeness.

In [46]:
# counts.reset_index(inplace=True) returns Cannot reset_index inplace on a Series to create a DataFrame 
rel_income_df =  counts.reset_index()
rel_income_df.head(10)

# We are working with a series, so melt doesn't work here!
# melt_df = pd.melt(counts, id_vars='religion', value_name='freq', var_name='income')
# melt_df.head(10)


,religion,income,0
0,Agnostic,$10-20k,34
1,Agnostic,$100-150k,109
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Agnostic,$75-100k,122
7,Agnostic,<$10k,27
8,Agnostic,>150k,84
9,Atheist,$10-20k,27


In [47]:
# As we extracted our data from a series of int64, all data types were lost along. We need to set the data types again.
rel_income_df['religion'] = rel_income_df['religion'].astype('category')
rel_income_df['income'] = rel_income_df['income'].astype('category')
rel_income_df.dtypes

religion    category
income      category
0              int64
dtype: object

In [48]:
# Then we rename to 0 column to freq
rel_income_df.rename(columns={0: 'freq'}, inplace=True)
rel_income_df.head()

,religion,income,freq
0,Agnostic,$10-20k,34
1,Agnostic,$100-150k,109
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76


In [49]:
# Now we set the correct order for the income category
income_order_list = ['<$10k', '$10-20k', '$20-30k',	'$30-40k','$40-50k', '$50-75k', '$75-100k', '$100-150k', '>150k', "Don't know / refused"]
rel_income_df['income'] = rel_income_df['income'].cat.set_categories(income_order_list, ordered=True)
# Without the following line, category is ordered but the dataframe is not. So we need to sort values inplace.
rel_income_df.sort_values(['religion','income'], inplace=True)
rel_income_df.head(10)


,religion,income,freq
7,Agnostic,<$10k,27
0,Agnostic,$10-20k,34
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Agnostic,$75-100k,122
1,Agnostic,$100-150k,109
8,Agnostic,>150k,84
16,Atheist,<$10k,12


In [50]:
# To make it event tidier, let's reset the index to the current order of rows.
rel_income_df.reset_index(drop=True, inplace=True)
rel_income_df.head(10)

,religion,income,freq
0,Agnostic,<$10k,27
1,Agnostic,$10-20k,34
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Agnostic,$75-100k,122
7,Agnostic,$100-150k,109
8,Agnostic,>150k,84
9,Atheist,<$10k,12


## Multiple variables are stored in one column

In this example we will work with a dataset about tuberculosis from WHO.

### Load

In [51]:
# Import from csv
raw = pd.read_csv('https://raw.github.com/hadley/tidy-data/master/data/tb.csv', na_values="")
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5769 entries, 0 to 5768
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   iso2          5746 non-null   object 
 1   year          5769 non-null   int64  
 2   new_sp        3081 non-null   float64
 3   new_sp_m04    392 non-null    float64
 4   new_sp_m514   401 non-null    float64
 5   new_sp_m014   2381 non-null   float64
 6   new_sp_m1524  2407 non-null   float64
 7   new_sp_m2534  2408 non-null   float64
 8   new_sp_m3544  2415 non-null   float64
 9   new_sp_m4554  2421 non-null   float64
 10  new_sp_m5564  2414 non-null   float64
 11  new_sp_m65    2408 non-null   float64
 12  new_sp_mu     273 non-null    float64
 13  new_sp_f04    393 non-null    float64
 14  new_sp_f514   404 non-null    float64
 15  new_sp_f014   2378 non-null   float64
 16  new_sp_f1524  2394 non-null   float64
 17  new_sp_f2534  2400 non-null   float64
 18  new_sp_f3544  2398 non-null 

In [52]:
# Remove column new_sp
# In the original paper, it was filled with NULLs: raw$new_sp <- NULL
raw.drop(columns=['new_sp'], inplace=True)

In [53]:
# Rename first colum as 'country'
# R original: command names(raw)[1] <- "country"
raw.rename(columns={'iso2': 'country'}, inplace=True)

In [54]:
# Remove new_sp_ from every column name
# R original: names(raw) <- str_replace(names(raw), "new_sp_", "")
# Alternative command in Python
# raw.columns = raw.columns.str.replace('new_sp_','')
raw.columns = raw.columns.str.lstrip('new_sp_')

In [55]:
# subset year 2000 results
# R original: raw <- subset(raw, year == 2000)
raw = raw[raw['year'] == 2000]

In [56]:
# Set m04, m514, f04, f514 columns to nan
import numpy as np
raw['m04'] = np.nan
raw['m514'] = np.nan
raw['f04'] = np.nan
raw['f514'] = np.nan
raw.head(10)

,country,year,m04,m514,m014,m1524,m2534,m3544,m4554,m5564,...,f04,f514,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
10,AD,2000,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,AE,2000,NaN,NaN,2.0,4.0,4.0,6.0,5.0,12.0,...,NaN,NaN,3.0,16.0,1.0,3.0,0.0,0.0,4.0,NaN
60,AF,2000,NaN,NaN,52.0,228.0,183.0,149.0,129.0,94.0,...,NaN,NaN,93.0,414.0,565.0,339.0,205.0,99.0,36.0,NaN
87,AG,2000,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN
136,AL,2000,NaN,NaN,2.0,19.0,21.0,14.0,24.0,19.0,...,NaN,NaN,3.0,11.0,10.0,8.0,8.0,5.0,11.0,NaN
165,AM,2000,NaN,NaN,2.0,152.0,130.0,131.0,63.0,26.0,...,NaN,NaN,1.0,24.0,27.0,24.0,8.0,8.0,4.0,NaN
178,AN,2000,NaN,NaN,0.0,0.0,1.0,2.0,0.0,0.0,...,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,NaN
207,AO,2000,NaN,NaN,186.0,999.0,1003.0,912.0,482.0,312.0,...,NaN,NaN,247.0,1142.0,1091.0,844.0,417.0,200.0,120.0,NaN
236,AR,2000,NaN,NaN,97.0,278.0,594.0,402.0,419.0,368.0,...,NaN,NaN,121.0,544.0,479.0,262.0,230.0,179.0,216.0,NaN
265,AS,2000,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [57]:
# R melt has a na.rm parameter that removes columns full of nan. Given Python doesn't have that option we will just drop the columns
raw.drop(columns=['m04','m514', 'f04', 'f514'], inplace=True)
raw.head(10)

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
10,AD,2000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,AE,2000,2.0,4.0,4.0,6.0,5.0,12.0,10.0,NaN,3.0,16.0,1.0,3.0,0.0,0.0,4.0,NaN
60,AF,2000,52.0,228.0,183.0,149.0,129.0,94.0,80.0,NaN,93.0,414.0,565.0,339.0,205.0,99.0,36.0,NaN
87,AG,2000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN
136,AL,2000,2.0,19.0,21.0,14.0,24.0,19.0,16.0,NaN,3.0,11.0,10.0,8.0,8.0,5.0,11.0,NaN
165,AM,2000,2.0,152.0,130.0,131.0,63.0,26.0,21.0,NaN,1.0,24.0,27.0,24.0,8.0,8.0,4.0,NaN
178,AN,2000,0.0,0.0,1.0,2.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,NaN
207,AO,2000,186.0,999.0,1003.0,912.0,482.0,312.0,194.0,NaN,247.0,1142.0,1091.0,844.0,417.0,200.0,120.0,NaN
236,AR,2000,97.0,278.0,594.0,402.0,419.0,368.0,330.0,NaN,121.0,544.0,479.0,262.0,230.0,179.0,216.0,NaN
265,AS,2000,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


This table is the starting point for the melt process in the original paper. Let's go!

### Melt
Melting the table will take all the columns associated with our groups and convert them into a single column.

In [58]:
# Melt operation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.melt.html
# R Original: clean <- melt(raw, id = c("country", "year"), na.rm = TRUE)
# Note the na.rm = TRUE parameter, not available in pandas melt. We will clear the NaN later.

clean = raw.melt(id_vars=['country', 'year'])
clean.head(10)

,country,year,variable,value
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0
5,AM,2000,m014,2.0
6,AN,2000,m014,0.0
7,AO,2000,m014,186.0
8,AR,2000,m014,97.0
9,AS,2000,m014,NaN


In [59]:
# Now we rename the columns. following the original instructions
# names(clean)[3] <- "column"
# names(clean)[4] <- "cases"
clean.rename(columns={'variable':'column', 'value':'cases'}, inplace=True)
clean.head(10)

,country,year,column,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0
5,AM,2000,m014,2.0
6,AN,2000,m014,0.0
7,AO,2000,m014,186.0
8,AR,2000,m014,97.0
9,AS,2000,m014,NaN


In [60]:
# To obtain a similar table to table 10 (a) Molten data, we need to sort our dataframe by country, column ascending
clean.sort_values(by=['country','column'], inplace=True)
clean.head(10)

,country,year,column,cases
1608,AD,2000,f014,NaN
1809,AD,2000,f1524,NaN
2010,AD,2000,f2534,NaN
2211,AD,2000,f3544,NaN
2412,AD,2000,f4554,NaN
2613,AD,2000,f5564,NaN
2814,AD,2000,f65,NaN
3015,AD,2000,fu,NaN
0,AD,2000,m014,0.0
201,AD,2000,m1524,0.0


In [61]:
# Note that the output is not exactly the same. The reason is that R melt dropped all NaN. 
# For example, all AD country females data is missing. Total cases are not zero, it's just that the data is missing.
# In our case, we need to dropna to get the same results. Also we, need to convert cases from float to int.

clean.dropna(inplace=True)
clean['cases'] = clean['cases'].astype('Int64')
clean.head(15)

,country,year,column,cases
0,AD,2000,m014,0
201,AD,2000,m1524,0
402,AD,2000,m2534,1
603,AD,2000,m3544,0
804,AD,2000,m4554,0
1005,AD,2000,m5564,0
1206,AD,2000,m65,0
1609,AE,2000,f014,3
1810,AE,2000,f1524,16
2011,AE,2000,f2534,1


#### Melt discussion
In our example, we have just replicated the commands from the original paper. Some of the options of R command line na.rm are not available in Pandas. On the other hand, Panda's melt() accepts parameters such as variable_name and value_name, that would make renaming the columns afterwards unnecessary.

### Tidy data
In this last step, we will split clean['column'] into the two different variables it includes: age and sex. This will make the data more explicit and make the analysis simpler.

In [62]:
# Extract the first character and populate column sex
# R original command: clean$sex <- str_sub(clean$column, 1, 1)
# Given there is no delimiter, we will use Python string slicing

clean['sex'] = clean['column'].str[0]
clean['age'] = clean['column'].str[1:]
clean.head()

,country,year,column,cases,sex,age
0,AD,2000,m014,0,m,014
201,AD,2000,m1524,0,m,1524
402,AD,2000,m2534,1,m,2534
603,AD,2000,m3544,0,m,3544
804,AD,2000,m4554,0,m,4554


In [63]:
# Create a mapping to replace remaining values into age groups
# R original command: ages <- c("04" = "0-4", "514" = "5-14", "014" = "0-14", "1524" = "15-24", "2534" = "25-34", "3544" = "35-44", "4554" = "45-54", "5564" = "55-64", "65"= "65+", "u" = NA)
age_groups  = {
    "04": "0-4",
    "514": "5-14", 
    "014": "0-14",
    "1524": "15-24", 
    "2534": "25-34", 
    "3544": "35-44", 
    "4554": "45-54", 
    "5564": "55-64", 
    "65": "65+", 
    "u": "Unknown"
}

In [64]:
# Create new ordered categorical data column using the previous mapping. Sort the columns
# R original commands: clean$age <- factor(ages[str_sub(clean$column, 2)], levels = ages)
clean['age'] = clean['age'].map(age_groups)

# We drop the column 'column' as we have already split it in two
clean.drop(columns=['column'], inplace=True)
clean.head()

,country,year,cases,sex,age
0,AD,2000,0,m,0-14
201,AD,2000,0,m,15-24
402,AD,2000,1,m,25-34
603,AD,2000,0,m,35-44
804,AD,2000,0,m,45-54


In [65]:
# clean <- clean[c("country", "year", "sex", "age", "cases")]
# Now we rearrange the columns
# Refence: https://stackoverflow.com/a/47467999

columns_order = ['country', 'year', 'sex', 'age', 'cases']
result = clean.reindex(columns = columns_order)
result.head(15)


,country,year,sex,age,cases
0,AD,2000,m,0-14,0
201,AD,2000,m,15-24,0
402,AD,2000,m,25-34,1
603,AD,2000,m,35-44,0
804,AD,2000,m,45-54,0
1005,AD,2000,m,55-64,0
1206,AD,2000,m,65+,0
1609,AE,2000,f,0-14,3
1810,AE,2000,f,15-24,16
2011,AE,2000,f,25-34,1


## Variables are stored in both rows and columns

## Multiple types of observational units are stored in the same table

## A single observational unit is stored in multiple tables

# Fundamental operations
There are three fundamental operations:
- Pivot
- Stack / Unstack
- Melt

Reference: https://pandas.pydata.org/docs/user_guide/reshaping.html

Reference: https://wesmckinney.com/book/data-wrangling.html#prep_reshape